In [74]:
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import numpy as np
import glob

In [67]:
era5_precip = xr.open_dataset('../era5_raw_data/era5_precip.nc').tp.sel(time=slice('1980-01-01','2014-12-31')).mean(['longitude'])*1000

era5_precip_trend = era5_precip.groupby('time.year').mean('time').polyfit(dim='year', deg=1)


gpcp_precip = xr.open_dataset('../precip.mon.mean.nc').precip.sel(time=slice('1980-01-01','2014-12-31')).mean(['lon'])
gpcp_precip_trend = gpcp_precip.groupby('time.year').mean('time').polyfit(dim='year', deg=1)



In [68]:
merra_precip = xr.open_mfdataset('../merra_data/merra_precip/*.nc4').PRECTOT.sel(time=slice('1980-01-01','2014-12-31')).mean(['lon'])*86400
merra_precip_trend = merra_precip.groupby('time.year').mean('time').polyfit(dim='year', deg=1)

merra_precip['time'] = era5_precip.time

merra_precip = merra_precip.interp(lat=era5_precip.latitude)


In [76]:
jra_datas = []

jra_files = glob.glob('../JRA_aht/raw_monthly_data/precip/*')

for file in jra_files:
    
    jra_precip1 = xr.open_dataset(file)
    jra_precip1 = jra_precip1.rename({'g0_lat_1':'lat',
                                   'g0_lon_2':'lon',
                                   'initial_time0_hours':'time',
                                   'TPRAT_GDS0_SFC_S130':'pr'})

    jra_precip1 = jra_precip1.pr.mean(['lon'])
    jra_precip1 = jra_precip1.interp(lat=era5_precip.latitude)
    
    jra_datas.append(jra_precip1)
    
    
jra_precip = xr.concat(jra_datas, dim='time')

jra_precip = jra_precip.sel(time=slice('1980-01-01','2014-12-31'))
jra_precip['time'] = era5_precip.time

In [77]:
reanalysis_precip = xr.concat([era5_precip, jra_precip, merra_precip],
                              pd.Index(['ERA5', 'JRA', 'MERRA'], name='model'))

reanalysis_precip = reanalysis_precip.compute()

#reanalysis_precip.to_netcdf('../AHT_trends_work/reanalysis_precip.nc')

In [4]:
models = ["AWI-CM-1-1-MR",
"AWI-ESM-1-1-LR",
"BCC-CSM2-MR",
"BCC-ESM1",
"CAMS-CSM1-0",
"CAS-ESM2-0",
"CIESM",
"CMCC-ESM2",
"CanESM5-1",
"E3SM-1-1",
"E3SM-1-1-ECA",
"E3SM-2-0",
"EC-Earth3-Veg-LR",
"FIO-ESM-2-0",
"GFDL-ESM4",
"GISS-E2-2-H",
"ICON-ESM-LR",
"IPSL-CM5A2-INCA",
"IPSL-CM6A-LR-INCA",
"KIOST-ESM",
"MCM-UA-1-0",
"NorESM2-MM",
    "ACCESS-CM2",
    "ACCESS-ESM1-5",
    "CanESM5",
    "CESM-FV2",
    "CESM2",
    "CESM2-WACCM",
    "CESM2-WACCM-FV2",
    "CMCC-CM2-HR4",
    "CMCC-CM2-SR5",
    "CNRM-CM6-1-HR",
    "CNRM-CM6-1",
    "E3SM-1-0",
    "EC-Earth3-AerChem",
    "EC-Earth3-CC",
    "EC-Earth3-Veg",
    "EC-Earth3",
    "FGOALS-f3-L",
    "FGOALS-g3",
    "GFDL-CM4",
    "GISS-E2-1-G",
    "GISS-E2-2-G",
    "GISS-E2-1-H",
    "HadGEM3-GC31-LL",
    "HadGEM3-GC31-MM",
    "IITM-ESM",
    "INM-CM4-8",
    "INM-CM5-0",
    "IPSL-CM6A-LR",
    "KACE-1-0-G",
    "MIROC6",
    "MIROC-ES2L",
    "MPI-ESM-1-2-HAM",
    "MPI-ESM1-2-HR",
    "MPI-ESM1-2-LR",
    "MRI-ESM2-0",
    "NESM3",
    "NorCPM1",
    "NorESM2-LM",
    "SAM0-UNICON",
    "TaiESM1",
    "UKESM1-0-LL"
]

In [45]:
def open_pr(file):

    final_times = pd.date_range('1980-01-01', '2014-12-31', freq='1M')
    
    final_lat = np.arange(-90, 91, 1)
    
    pr_ds = xr.open_mfdataset(file).pr.mean('lon')
    
    pr_ds = pr_ds.sel(time=slice('1980-01-01','2014-12-31'))

    pr_ds = pr_ds.interp(lat=final_lat)
    
    pr_ds['time'] = final_times
    
    return pr_ds

In [46]:
prs_amip = []
prs_amip_names = []
prs_historical = []
prs_historical_names = []
for model in models:
    try:   
        file_name = f'../CMIP6/data/s_{model}_e_historical_vl_r1i1p1f1_f_mon_r_atmos_v_pr/*Amon*'
        precip_historical = open_pr(file_name)
        prs_historical.append(precip_historical)
        prs_historical_names.append(model)
    except:
        print('Historical: ' + model)
      
    try:
        file_name = f'../CMIP6/data/s_{model}_e_amip_vl_r1i1p1f1_f_mon_r_atmos_v_pr/*Amon*'
        precip_amip = open_pr(file_name)
        prs_amip.append(precip_amip)
        prs_amip_names.append(model)
        
    except:
        print('AMIP: ' + model)

    

AMIP: AWI-CM-1-1-MR
AMIP: AWI-ESM-1-1-LR
Historical: CIESM
AMIP: CIESM
AMIP: CMCC-ESM2
AMIP: CanESM5-1
AMIP: E3SM-1-1
AMIP: E3SM-1-1-ECA
Historical: EC-Earth3-Veg-LR
AMIP: GISS-E2-2-H
Historical: ICON-ESM-LR
AMIP: ICON-ESM-LR
AMIP: IPSL-CM5A2-INCA
AMIP: IPSL-CM6A-LR-INCA
Historical: MCM-UA-1-0
AMIP: MCM-UA-1-0
AMIP: NorESM2-MM
Historical: ACCESS-CM2
Historical: CESM-FV2
AMIP: CESM-FV2


/tdat/tylersc/mini3/envs/jupyter_new/lib/python3.9/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'pr' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/tdat/tylersc/mini3/envs/jupyter_new/lib/python3.9/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'pr' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/tdat/tylersc/mini3/envs/jupyter_new/lib/python3.9/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'pr' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/tdat/tylersc/mini3/envs/jupyter_new/lib/python3.9/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'pr' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/tdat/tylersc/mini3/envs/jupyter_new/lib/python3.9/site-packages/xar

Historical: CNRM-CM6-1-HR
AMIP: CNRM-CM6-1-HR
Historical: CNRM-CM6-1
AMIP: CNRM-CM6-1
Historical: EC-Earth3-AerChem
Historical: GFDL-CM4
AMIP: GFDL-CM4
AMIP: GISS-E2-2-G
AMIP: GISS-E2-1-H
Historical: HadGEM3-GC31-LL
AMIP: HadGEM3-GC31-LL
Historical: HadGEM3-GC31-MM
AMIP: HadGEM3-GC31-MM
Historical: KACE-1-0-G
AMIP: KACE-1-0-G
Historical: MIROC-ES2L
AMIP: MIROC-ES2L
Historical: SAM0-UNICON
AMIP: SAM0-UNICON
Historical: UKESM1-0-LL
AMIP: UKESM1-0-LL


In [47]:
hist_pr_ds = xr.concat(prs_historical,  pd.Index(prs_historical_names, name='model'))
amip_pr_ds = xr.concat(prs_amip, pd.Index(prs_amip_names, name='model'))


In [49]:
#hist_pr_ds.to_netcdf('../AHT_trends_work/hist_pr_ds.nc')
#amip_pr_ds.to_netcdf('../AHT_trends_work/amip_pr_ds.nc')